In [1]:
import os
dll_directory = r'C:\Users\beyza\anaconda3\Lib\site-packages\openslide\openslide-win64-20230414\bin'
os.add_dll_directory(dll_directory)

<AddedDllDirectory('C:\\Users\\beyza\\anaconda3\\Lib\\site-packages\\openslide\\openslide-win64-20230414\\bin')>

In [2]:
import openslide
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import csv
import xml.etree.ElementTree as ET
import cv2
from PIL import Image, ImageFile
from openslide import OpenSlide
from patchify import patchify
from PIL.Image import DecompressionBombWarning
import warnings

In [3]:
file_paths = [
    "C:/Users/beyza/Desktop/AIN3007_project/train_valid_data/Breast1__he/",
    "C:/Users/beyza/Desktop/AIN3007_project/train_valid_data/Breast2__he/",
    "C:/Users/beyza/Desktop/AIN3007_project/train_valid_data/Breast3__ihc/",
    "C:/Users/beyza/Desktop/AIN3007_project/train_valid_data/LymphNode1__he/",
    "C:/Users/beyza/Desktop/AIN3007_project/train_valid_data/Prostate1__he/",
]

all_slides = []
all_annotations = []

for file_path in file_paths:
    # List all files in file
    for root, dirs, files in os.walk(file_path):
        for file in files:
            # Check the extension of the file.
            if file.lower().endswith((".svs", ".tiff", "tif", ".ndpi", ".mrxs")):
                # Save the slides path in all_slides.
                file_full_path = os.path.join(root, file)
                all_slides.append(f'{file_full_path}')

for file_path in file_paths:
    # List all files in file
    for root, dirs, files in os.walk(file_path):
        for file in files:
            # Check the extension of the file.
            if file.lower().endswith((".xml")):
                # Save the slides path in all_slides.
                file_full_path = os.path.join(root, file)
                all_annotations.append(f'{file_full_path}')

# Specify the output file paths
annotations_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\all_annotations.txt"
slides_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\all_slides.txt"

with open(slides_file_path, "w") as slides_file:
    for slide_path in all_slides:
        slides_file.write(slide_path + "\n")

with open(annotations_file_path, "w") as annotations_file:
    for annotation_path in all_annotations:
        annotations_file.write(annotation_path + "\n")


In [4]:
# Dosya yollarını tanımlayın
annotations_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\all_annotations.txt"
slides_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\all_slides.txt"

# Verileri dosyalardan okuyun
with open(annotations_file_path, "r") as annotations_file:
    annotations_data = annotations_file.readlines()

with open(slides_file_path, "r") as slides_file:
    slides_data = slides_file.readlines()

# CSV dosyasına yazmak için yeni bir dosya açın ve verileri yazın
csv_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\annotations_and_slides.csv"

# Verileri birleştirin
combined_data = zip(slides_data, annotations_data)

# CSV dosyasına verileri yazın
with open(csv_file_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Slides", "Annotations"])  # Başlık satırını yazın (isteğe bağlı)
    csv_writer.writerows(combined_data)

print(f"CSV dosyası başarıyla oluşturuldu: {csv_file_path}")


CSV dosyası başarıyla oluşturuldu: C:\Users\beyza\Desktop\AIN3007_project\annotations_and_slides.csv


In [5]:
df = pd.read_csv(csv_file_path)
df.head()

,Slides,Annotations
0,C:/Users/beyza/Desktop/AIN3007_project/train_v...,C:/Users/beyza/Desktop/AIN3007_project/train_v...
1,C:/Users/beyza/Desktop/AIN3007_project/train_v...,C:/Users/beyza/Desktop/AIN3007_project/train_v...
2,C:/Users/beyza/Desktop/AIN3007_project/train_v...,C:/Users/beyza/Desktop/AIN3007_project/train_v...
3,C:/Users/beyza/Desktop/AIN3007_project/train_v...,C:/Users/beyza/Desktop/AIN3007_project/train_v...
4,C:/Users/beyza/Desktop/AIN3007_project/train_v...,C:/Users/beyza/Desktop/AIN3007_project/train_v...


In [6]:
df = pd.read_csv("C:\\Users\\beyza\\Desktop\\AIN3007_project\\all_slides.txt", header=None, names=['Slides'])

# Adım 2: Slaytlar üzerinde döngü yapın ve boyutları alın
for index, row in df.iterrows():
    wsi_path = row['Slides']
    try:
        slide = openslide.OpenSlide(wsi_path)
        level_dims = slide.level_dimensions
        first_level_dims = level_dims[0]  # İlk seviye boyutlarını al
        df.at[index, 'image_shape'] = str(first_level_dims)
    except Exception as e:
        print(f"Error opening {wsi_path}: {e}")
        df.at[index, 'image_shape'] = "Error"

# Adım 3: Sonuçları CSV dosyasına yazdırın
df.to_csv(r"C:\Users\beyza\Desktop\AIN3007_project\annotations_and_slides.csv", index=False)
df.shape

Error opening C:/Users/beyza/Desktop/AIN3007_project/train_valid_data/LymphNode1__he/patient_044_node_0.tif: Unsupported or missing image file
Error opening C:/Users/beyza/Desktop/AIN3007_project/train_valid_data/LymphNode1__he/patient_069_node_0.tif: Unsupported or missing image file


(200, 2)

In [7]:
with open("C:\\Users\\beyza\\Desktop\\AIN3007_project\\all_annotations.txt", "r") as file:
    annotations = file.readlines()  # Her satırı bir liste elemanı olarak okur.

# Satır sonu karakterlerini kaldırın (\n)
annotations = [line.strip() for line in annotations]
print(len(annotations))
# Annotations listesini DataFrame'e 'annotations' adlı yeni bir sütun olarak ekleyin.
df['annotations'] = annotations
# Güncellenmiş DataFrame'i CSV olarak kaydedin.
df.to_csv(r'C:\Users\beyza\Desktop\AIN3007_project\annotations_and_slides.csv', index=False)
df.shape

200


(200, 3)

In [8]:
df.head()

,Slides,image_shape,annotations
0,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(100794, 80501)",C:/Users/beyza/Desktop/AIN3007_project/train_v...
1,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(159380, 73062)",C:/Users/beyza/Desktop/AIN3007_project/train_v...
2,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(70448, 76478)",C:/Users/beyza/Desktop/AIN3007_project/train_v...
3,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(95200, 47660)",C:/Users/beyza/Desktop/AIN3007_project/train_v...
4,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(116144, 40272)",C:/Users/beyza/Desktop/AIN3007_project/train_v...


In [9]:
for index, row in df.iterrows():
    image_shape = row['image_shape']
    if image_shape.startswith("(") and image_shape.endswith(")"):
        image_shape = image_shape[1:-1]  # Parantezleri kaldır
        dimensions = image_shape.split(',')
        if len(dimensions) == 2:
            dimensions[0], dimensions[1] = dimensions[1], dimensions[0]  # Boyutları değiştir
            new_image_shape = f"({dimensions[0]}, {dimensions[1]})"
            df.at[index, 'image_shape'] = new_image_shape

def process_image_shape(x):
    try:
        # Parantezleri ve boşlukları kaldırın, sonra virgüllerle ayırın
        parts = x.strip('()').split(',')
        # Her iki parçayı da tamsayıya dönüştürün
        return tuple(map(int, parts))
    except ValueError:
        # Hata durumunda, None döndürün veya başka bir işlem yapın
        return None

# image_shape sütununu işleyin ve hatalı girdileri filtreleyin
df['image_shape'] = df['image_shape'].apply(process_image_shape)

df.to_csv(r'C:\Users\beyza\Desktop\AIN3007_project\annotations_and_slides.csv', index=False)
df.head()

,Slides,image_shape,annotations
0,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(80501, 100794)",C:/Users/beyza/Desktop/AIN3007_project/train_v...
1,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(73062, 159380)",C:/Users/beyza/Desktop/AIN3007_project/train_v...
2,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(76478, 70448)",C:/Users/beyza/Desktop/AIN3007_project/train_v...
3,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(47660, 95200)",C:/Users/beyza/Desktop/AIN3007_project/train_v...
4,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(40272, 116144)",C:/Users/beyza/Desktop/AIN3007_project/train_v...


In [10]:
import xml.etree.ElementTree as ET

def read_xml(xml_file_path):
    # Parse the XML file
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    return root

def extract_coordinates(xml_root):
    # Initialize dictionaries to hold coordinates for each group
    groups = {
        "roi": [],
        "tissue": [],
        "bg": []
    }
    
    # Iterate through all the 'Annotation' elements
    for annotation in xml_root.findall('.//Annotation'):
        part_of_group = annotation.get('PartOfGroup')  # Get the group name
        coords = []
        
        # For each 'Coordinate' in 'Coordinates', extract 'X' and 'Y'
        for coordinate in annotation.find('Coordinates'):
            x = float(coordinate.get('X').replace(',', '.'))  # Ensure number format
            y = float(coordinate.get('Y').replace(',', '.'))
            coords.append((x, y))
        
        # Add the coordinate list to the appropriate group based on 'PartOfGroup'
        if part_of_group in groups:
            groups[part_of_group].append(coords)
    
    return groups

In [11]:
df['coords_list'] = None 
df['bg_coords'] = None
df['roi_coords'] = None
# Loop through the specified rows of the DataFrame
for index, row in df.iterrows():
    xml_file_path = row["annotations"]  # Get the path to the XML file
    xml_root = read_xml(xml_file_path)  # Read the XML file
    grouped_coordinates = extract_coordinates(xml_root)  # Extract and group coordinates

    # Assign the extracted coordinates to the DataFrame
    df.at[index, 'roi_coords'] = grouped_coordinates["roi"]
    df.at[index, 'coords_list'] = grouped_coordinates["tissue"]
    df.at[index, 'bg_coords'] = grouped_coordinates["bg"]    
df.to_csv(r'C:\Users\beyza\Desktop\AIN3007_project\annotations_and_slides.csv', index=False)    


In [12]:
df.head()

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords
0,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(80501, 100794)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(2171.54419, 8883.58984), (10068.0684, 6712....",[],[]
1,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(73062, 159380)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(8585.2832, 35682.582), (10999.8936, 30853.3...",[],[]
2,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(76478, 70448)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(3381.96362, 34562.0195), (3546.93726, 31097...",[],[]
3,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(47660, 95200)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(10470.8252, 33363.8438), (9814.84668, 32569...",[],[]
4,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(40272, 116144)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(3443.17017, 19903.2012), (2687.35229, 18223...",[],[]


In [13]:
def create_binary_mask(roi_coords, tissue_coords, bg_coords, image_shape):
    # Ensure image_shape is a tuple of integers
    image_shape = tuple(map(int, image_shape))

    # Define the image shape based on roi_coords
    if roi_coords:
        top_left = roi_coords[0][0]  # (x_min, y_min)
        bottom_right = roi_coords[0][2]  # (x_max, y_max)
        image_shape = (int(bottom_right[1] - top_left[1]), int(bottom_right[0] - top_left[0]))  # (height, width)
    else:
        top_left = (0, 0)  # (x_min, y_min)
        bottom_right = (image_shape[1], image_shape[0])  # (x_max, y_max)
        image_shape = (int(bottom_right[1] - top_left[1]), int(bottom_right[0] - top_left[0]))  # (height, width)

    # Rest of your code remains the same

    # Initialize an all-white image (background)
    binary_mask = np.full(image_shape, 255, dtype=np.uint8)

    # Draw the 'tissue' areas in black
    for coords in tissue_coords:
        # Convert coordinates to relative positions within the ROI
        relative_coords = [(int(x - top_left[0]), int(y - top_left[1])) for x, y in coords]
        pts = np.array(relative_coords, np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(binary_mask, [pts], 0)  # 0 for black

    # Draw the 'bg' areas in white
    for coords in bg_coords:
        # Convert coordinates to relative positions within the ROI
        relative_coords = [(int(x - top_left[0]), int(y - top_left[1])) for x, y in coords]
        pts = np.array(relative_coords, np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(binary_mask, [pts], 255)  # 255 for white

    return binary_mask

In [15]:
output_dir = 'C:\\Users\\beyza\\Desktop\\AIN3007_project\\mask'
os.makedirs(output_dir, exist_ok=True)

# Iterate through the specified range of the DataFrame
for index, row in df.iloc[65:68].iterrows():
    # Extract coordinates from the DataFrame row
    roi_coords = row['roi_coords']
    tissue_coords = row['coords_list']
    bg_coords = row['bg_coords']
    image_shape = row['image_shape']
   
   # Create the binary mask
    binary_mask = create_binary_mask(roi_coords, tissue_coords, bg_coords, image_shape)
    
    # Define the path to save the binary mask
    mask_filename = os.path.join(output_dir, f'mask_{index}.png')
    cv2.imwrite(mask_filename, binary_mask)
    print(f"Mask for row {index} saved.")

KeyboardInterrupt: 

In [14]:
mask_folder_path = "C:/Users/beyza/Desktop/AIN3007_project/mask/"

# Mask dosyalarının sayısı
num_masks = len(df)

# Mask dosyalarının yollarını eklemek için bir liste oluşturma
mask_paths = [os.path.join(mask_folder_path, f"mask_{i}.png") for i in range(num_masks)]

# Yeni "mask_path" sütununu veri çerçevesine ekleme
df["mask_path"] = mask_paths
df.to_csv(r'C:\Users\beyza\Desktop\AIN3007_project\annotations_and_slides.csv', index=False)    
df.head()

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords,mask_path
0,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(80501, 100794)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(2171.54419, 8883.58984), (10068.0684, 6712....",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
1,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(73062, 159380)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(8585.2832, 35682.582), (10999.8936, 30853.3...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
2,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(76478, 70448)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(3381.96362, 34562.0195), (3546.93726, 31097...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
3,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(47660, 95200)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(10470.8252, 33363.8438), (9814.84668, 32569...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
4,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(40272, 116144)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(3443.17017, 19903.2012), (2687.35229, 18223...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...


In [15]:
df.iloc[0:40]

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords,mask_path
0,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(80501, 100794)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(2171.54419, 8883.58984), (10068.0684, 6712....",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
1,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(73062, 159380)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(8585.2832, 35682.582), (10999.8936, 30853.3...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
2,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(76478, 70448)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(3381.96362, 34562.0195), (3546.93726, 31097...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
3,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(47660, 95200)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(10470.8252, 33363.8438), (9814.84668, 32569...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
4,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(40272, 116144)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(3443.17017, 19903.2012), (2687.35229, 18223...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
5,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(53504, 45312)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(8138.14893, 37689.4414), (8946.19238, 36015...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
6,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(33536, 38144)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(17966.2754, 29506.6562), (18818.6895, 29637...","[[(15292.0225, 20498.9824), (15027.1064, 20197...",[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
7,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(86784, 53504)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(20746.3438, 66768.1641), (21769.0508, 64576...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
8,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(53300, 67592)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(531.0, 35648.0), (4090.0, 33682.0), (5100.0...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
9,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(57504, 36451)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(62.037468, 7134.30859), (496.299713, 7320.4...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...


In [16]:
df.iloc[40:80]

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords,mask_path
40,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(189030.219, 177975.281), (189115.828, 17841...",[],"[[(158747.219, 118328.703), (258539.141, 11832...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
41,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(171745.203, 157122.188), (171758.062, 15718...","[[(176810.812, 134501.438), (176813.031, 13459...","[[(157121.422, 41691.0703), (273969.875, 41691...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
42,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(8284.51367, 73894.6406), (8341.9248, 73995....",[],"[[(1199.92578, 39997.6172), (109993.453, 39997...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
43,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(229025.203, 162392.031), (229134.875, 16233...",[],"[[(176327.703, 97737.0625), (262139.578, 97737...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
44,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(166184.656, 171993.141), (166402.969, 17222...",[],"[[(159360.938, 52820.1953), (264401.094, 52820...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
45,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(218351.406, 139515.938), (218367.0, 139835....",[],"[[(165769.172, 30670.9453), (273847.719, 30670...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
46,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(208013.0, 82960.0), (210533.0, 81203.0), (2...",[],"[[(162550.5, 55198.1719), (263611.875, 55198.1...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
47,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(172783.391, 131055.375), (172763.375, 13089...",[],"[[(161466.969, 93284.4922), (260857.344, 93284...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
48,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(37007.6719, 107271.531), (36931.1094, 10765...",[],"[[(6241.88281, 59398.5469), (89601.1953, 59398...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
49,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(294038, 271950)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(233107.75, 133663.969), (233107.75, 134140....",[],"[[(163037.906, 52829.332), (270213.188, 52829....",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...


In [17]:
df.iloc[80:120]

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords,mask_path
80,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(56320, 192512)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(6999.14404, 29703.6855), (8492.86426, 28850...",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
81,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(98304, 106496)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(35496.6289, 30.8398151), (35712.5078, 616.7...","[[(90648.6953, 43815.7383), (90654.1953, 43827...",[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
82,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(57600, 118784)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(16962.7949, 7941.47656), (17105.8184, 7927....","[[(10241.6084, 23382.0703), (10257.2295, 23393...",[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
83,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(48640, 135168)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(12962.9697, 7936.51221), (14285.7217, 7771....",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
84,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(58624, 118784)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(4319.11572, 9321.25391), (4821.33838, 8678....",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
85,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(39680, 122880)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(9703.22949, 5076.34863), (10390.6514, 4203....",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
86,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(58368, 147456)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(11325.4082, 26462.2188), (11385.9609, 26470...","[[(13072.3721, 31115.8105), (13066.999, 31134....",[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
87,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(64768, 180224)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(12502.0029, 10655.1162), (13127.1035, 9689....",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
88,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(55808, 147456)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(8409.12598, 8932.97363), (9125.96973, 8050....",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
89,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(85504, 110592)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(14016.9971, 4049.18774), (14089.5723, 3971....",[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...


In [18]:
df.iloc[120:160]

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords,mask_path
120,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(197226, 96651)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(10049.4209, 68025.625), (10116.4268, 66844....","[[(26436.2695, 72734.1406), (26329.2383, 72619...","[[(4040.67578, 51040.1133), (115478.266, 51040...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
121,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(197226, 96651)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(9965.06738, 84423.9922), (10192.3984, 83401...","[[(62856.3828, 80309.1953), (62937.1367, 79299...","[[(-2065.54688, 38065.125), (100621.773, 38065...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
122,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(210579, 94968)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(30856.582, 67028.7969), (31407.5938, 65861....",[],"[[(-4776.0, 42074.3047), (81192.0391, 42074.30...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
123,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(197226, 96651)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(7006.16943, 90878.0938), (7747.20654, 92225...","[[(46138.4219, 79961.1719), (47435.5391, 79919...","[[(-3888.08594, 56720.3828), (98574.5391, 5672...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
124,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(210579, 94968)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(9122.08398, 106085.688), (9162.16699, 10558...",[],"[[(-25017.1562, 64362.3164), (97339.4766, 6436...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
125,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(197226, 96651)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(20007.9277, 75916.9688), (19545.0195, 76688...",[],"[[(9357.35352, 31474.7422), (73582.8359, 31474...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
126,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(210579, 94968)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(40423.2852, 108964.242), (38901.625, 109493...","[[(74495.875, 104151.539), (74756.1797, 104315...","[[(-6813.25391, 9408.78906), (102847.719, 9408...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
127,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(210579, 94968)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(38450.1758, 58692.5977), (39024.0586, 57492...",[],"[[(21150.8672, 48442.3125), (74824.0391, 48442...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
128,C:/Users/beyza/Desktop/AIN3007_project/train_v...,None,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(5008.33594, 23726.8555), (8366.6582, 21483....",[],"[[(414.9375, -138.316406), (113830.727, -138.3...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
129,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(93184, 102400)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,"[[(47608.1484, 80338.75), (43468.3086, 83443.6...",[],"[[(-1401.85938, -1802.39453), (192855.906, -18...",C:/Users/beyza/Desktop/AIN3007_project/mask/ma...


In [19]:
df.iloc[160:200]

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords,mask_path
160,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(29440, 27648)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
161,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(13312, 15360)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
162,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(28664, 23904)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
163,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(34560, 28672)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
164,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(32773, 23904)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
165,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(33024, 23552)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
166,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(28100, 23904)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
167,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(10052, 15936)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
168,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(22528, 31744)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...
169,C:/Users/beyza/Desktop/AIN3007_project/train_v...,"(19968, 16384)",C:/Users/beyza/Desktop/AIN3007_project/train_v...,[],[],[],C:/Users/beyza/Desktop/AIN3007_project/mask/ma...


In [20]:
wsi_path = 'C:/Users/beyza/Desktop/AIN3007_project/train_valid_data/Breast3__ihc/15_HER2.ndpi'
wsi = OpenSlide(wsi_path)

In [21]:

print('The number of levels in the pyramid: {}'.format(wsi.level_count))
print('The sizes at different levels:',wsi.level_dimensions)
print('Downsample factors at different levels:',wsi.level_downsamples)


The number of levels in the pyramid: 9
The sizes at different levels: ((122880, 39680), (61440, 19840), (30720, 9920), (15360, 4960), (7680, 2480), (3840, 1240), (1920, 620), (960, 310), (480, 155))
Downsample factors at different levels: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0)


In [22]:
from PIL import Image
import numpy as np
from openslide import OpenSlide
import os
from patchify import patchify

Image.MAX_IMAGE_PIXELS = None  

def crop_patches(wsi_path, mask_path):
    # Open the whole slide image and mask
    wsi = OpenSlide(wsi_path)
    mask = Image.open(mask_path)
    
    level = 6

    downsampled_image = wsi.read_region((0, 0), level, wsi.level_dimensions[level])

    # Convert the PIL images to numpy arrays
    image_array = np.array(downsampled_image)
    mask_array = np.array(mask)

    # Resize the mask to match the WSI's downsampled dimensions
    mask_array_resized = np.array(mask.resize(wsi.level_dimensions[level]))

    # Define patch size
    patch_size = 256

    # Calculate padding needed for both dimensions
    def calculate_padding(array, patch_size):
        pad_height = patch_size - (array.shape[0] % patch_size)
        pad_width = patch_size - (array.shape[1] % patch_size)
        return pad_height, pad_width

    image_pad_height, image_pad_width = calculate_padding(image_array, patch_size)
    mask_pad_height, mask_pad_width = calculate_padding(mask_array_resized, patch_size)

    # Pad the arrays
    image_array_padded = np.pad(image_array, ((0, image_pad_height), (0, image_pad_width), (0, 0)), mode='constant', constant_values=255)
    mask_array_padded = np.pad(mask_array_resized, ((0, mask_pad_height), (0, mask_pad_width)), mode='constant', constant_values=255)

    # Create patches for both image and mask
    image_patches = patchify(image_array_padded, (patch_size, patch_size, 3), step=patch_size)
    mask_patches = patchify(mask_array_padded, (patch_size, patch_size), step=patch_size)

    # Extract the filename part from the WSI path
    wsi_filename_part = os.path.basename(wsi_path).split('.')[0]

    # Specify the base directory for saving patches
    base_save_dir = "C:/Users/beyza/Desktop/AIN3007_project/patch_crop/Breast3__ihc/"
    image_dir = os.path.join(base_save_dir, f'image_{wsi_filename_part}')
    mask_dir = os.path.join(base_save_dir, f'mask_{wsi_filename_part}')

    # Ensure the directories for image and mask patches are created
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)

    # Save the patches
    for i in range(image_patches.shape[0]):
        for j in range(image_patches.shape[1]):
            # Save image patch
            image_patch = image_patches[i, j, 0, :, :, :]
            image_patch_img = Image.fromarray(image_patch)
            image_patch_path = os.path.join(image_dir, f'patch_{i}_{j}.png')
            image_patch_img.save(image_patch_path)

            # Save mask patch
            mask_patch = mask_patches[i, j, :, :]
            mask_patch_img = Image.fromarray(mask_patch)
            mask_patch_path = os.path.join(mask_dir, f'patch_{i}_{j}.png')
            mask_patch_img.save(mask_patch_path)


In [93]:
for index, row in df.iloc[80:81].iterrows():
    wsi_path = row["Slides"]
    mask_path = row["mask_path"]
    crop_patches(wsi_path, mask_path) 

In [23]:
from PIL import Image
import numpy as np
from openslide import OpenSlide
import os
from patchify import patchify

Image.MAX_IMAGE_PIXELS = None  

def find_appropriate_level(wsi, desired_patch_size, desired_patch_number):
    """
    WSI'daki en uygun seviyeyi bulur.
    desired_patch_size: Oluşturulacak patch'in piksel cinsinden boyutu
    desired_patch_number: Oluşturulmak istenen minimum patch sayısı
    """
    for level in range(wsi.level_count - 1, -1, -1):
        width, height = wsi.level_dimensions[level]
        if (width // desired_patch_size) * (height // desired_patch_size) >= desired_patch_number:
            return level
    return 0  # Eğer uygun seviye bulunamazsa en düşük seviyeyi kullan

def crop_patches(wsi_path, mask_path, desired_patch_size=256, desired_patch_number=100):
    wsi = OpenSlide(wsi_path)
    mask = Image.open(mask_path)
    
    # En uygun seviyeyi bul
    level = find_appropriate_level(wsi, desired_patch_size, desired_patch_number)

    downsampled_image = wsi.read_region((0, 0), level, wsi.level_dimensions[level])
    image_array = np.array(downsampled_image.convert("RGB"))
    mask_array = np.array(mask)

    # Resize the mask to match the WSI's downsampled dimensions
    mask_array_resized = np.array(mask.resize(wsi.level_dimensions[level]))

    # Define patch size
    patch_size = 256

    # Calculate padding needed for both dimensions
    def calculate_padding(array, patch_size):
        pad_height = patch_size - (array.shape[0] % patch_size)
        pad_width = patch_size - (array.shape[1] % patch_size)
        return pad_height, pad_width

    image_pad_height, image_pad_width = calculate_padding(image_array, patch_size)
    mask_pad_height, mask_pad_width = calculate_padding(mask_array_resized, patch_size)

    # Pad the arrays
    image_array_padded = np.pad(image_array, ((0, image_pad_height), (0, image_pad_width), (0, 0)), mode='constant', constant_values=255)
    mask_array_padded = np.pad(mask_array_resized, ((0, mask_pad_height), (0, mask_pad_width)), mode='constant', constant_values=255)

    # Create patches for both image and mask
    image_patches = patchify(image_array_padded, (patch_size, patch_size, 3), step=patch_size)
    mask_patches = patchify(mask_array_padded, (patch_size, patch_size), step=patch_size)

    # Extract the filename part from the WSI path
    wsi_filename_part = os.path.basename(wsi_path).split('.')[0]

    # Specify the base directory for saving patches
    base_save_dir = "C:/Users/beyza/Desktop/AIN3007_project/patch_crop/Breast3__ihc/"
    image_dir = os.path.join(base_save_dir, f'image_{wsi_filename_part}')
    mask_dir = os.path.join(base_save_dir, f'mask_{wsi_filename_part}')

    # Ensure the directories for image and mask patches are created
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)

    # Save the patches
    for i in range(image_patches.shape[0]):
        for j in range(image_patches.shape[1]):
            # Save image patch
            image_patch = image_patches[i, j, 0, :, :, :]
            image_patch_img = Image.fromarray(image_patch)
            image_patch_path = os.path.join(image_dir, f'patch_{i}_{j}.png')
            image_patch_img.save(image_patch_path)

            # Save mask patch
            mask_patch = mask_patches[i, j, :, :]
            mask_patch_img = Image.fromarray(mask_patch)
            mask_patch_path = os.path.join(mask_dir, f'patch_{i}_{j}.png')
            mask_patch_img.save(mask_patch_path)


In [26]:
from PIL import Image
import numpy as np
from openslide import OpenSlide
import os
from patchify import patchify

Image.MAX_IMAGE_PIXELS = None  

def find_appropriate_level(wsi, desired_patch_size, desired_patch_number):
    """
    WSI'daki en uygun seviyeyi bulur.
    desired_patch_size: Oluşturulacak patch'in piksel cinsinden boyutu
    desired_patch_number: Oluşturulmak istenen minimum patch sayısı
    """
    for level in range(wsi.level_count - 1, -1, -1):
        width, height = wsi.level_dimensions[level]
        if (width // desired_patch_size) * (height // desired_patch_size) >= desired_patch_number:
            return level
    return 0  
def calculate_padding(array, patch_size):
    pad_height = patch_size - (array.shape[0] % patch_size)
    pad_width = patch_size - (array.shape[1] % patch_size)
    return pad_height, pad_width

def crop_patches(wsi_path, mask_path, roi_coords, desired_patch_size=256, desired_patch_number=100):
    wsi = OpenSlide(wsi_path)
    mask = Image.open(mask_path)

    level = find_appropriate_level(wsi, desired_patch_size, desired_patch_number)
    
    top_left = (int(roi_coords[0][0][0]), int(roi_coords[0][0][1]))  # Convert to integer and ensure it's a tuple
    bottom_right = (int(roi_coords[0][2][0]), int(roi_coords[0][2][1]))

    region_width = bottom_right[0] - top_left[0]
    region_height = bottom_right[1] - top_left[1]

    downsampled_region = wsi.read_region(top_left, level, (region_width, region_height))

    region_array = np.array(downsampled_region.convert("RGB"))
    mask_region = mask.crop((top_left[0], top_left[1], bottom_right[0], bottom_right[1]))
    mask_region_array = np.array(mask_region)

    mask_region_array_resized = np.array(mask_region.resize((region_width, region_height)))

    image_pad_height, image_pad_width = calculate_padding(region_array, desired_patch_size)
    mask_pad_height, mask_pad_width = calculate_padding(mask_region_array_resized, desired_patch_size)

    region_array_padded = np.pad(region_array, ((0, image_pad_height), (0, image_pad_width), (0, 0)), mode='constant', constant_values=255)
    mask_region_array_padded = np.pad(mask_region_array_resized, ((0, mask_pad_height), (0, mask_pad_width)), mode='constant', constant_values=255)

    image_patches = patchify(region_array_padded, (desired_patch_size, desired_patch_size, 3), step=desired_patch_size)
    mask_patches = patchify(mask_region_array_padded, (desired_patch_size, desired_patch_size), step=desired_patch_size)

    wsi_filename_part = os.path.basename(wsi_path).split('.')[0]
    base_save_dir = "C:/Users/beyza/Desktop/AIN3007_project/patch_crop/Breast3__ihc/"
    image_dir = os.path.join(base_save_dir, f'image_{wsi_filename_part}')
    mask_dir = os.path.join(base_save_dir, f'mask_{wsi_filename_part}')

    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)

    for i in range(image_patches.shape[0]):
        for j in range(image_patches.shape[1]):
            image_patch = image_patches[i, j, 0, :, :, :]
            image_patch_img = Image.fromarray(image_patch)
            image_patch_path = os.path.join(image_dir, f'patch_{i}_{j}.png')
            image_patch_img.save(image_patch_path)

            mask_patch = mask_patches[i, j, :, :]
            mask_patch_img = Image.fromarray(mask_patch)
            mask_patch_path = os.path.join(mask_dir, f'patch_{i}_{j}.png')
            mask_patch_img.save(mask_patch_path)

In [29]:
from PIL import Image
import numpy as np
from openslide import OpenSlide
import os
from patchify import patchify

Image.MAX_IMAGE_PIXELS = None  

def find_appropriate_level(wsi, desired_patch_size, desired_patch_number):
    """
    Finds the most appropriate level in the WSI.
    desired_patch_size: Size of the patch to be created, in pixels
    desired_patch_number: Minimum number of patches desired
    """
    for level in range(wsi.level_count - 1, -1, -1):
        width, height = wsi.level_dimensions[level]
        if (width // desired_patch_size) * (height // desired_patch_size) >= desired_patch_number:
            return level
    return 0  

def calculate_padding(array, patch_size):
    pad_height = patch_size - (array.shape[0] % patch_size)
    pad_width = patch_size - (array.shape[1] % patch_size)
    return pad_height, pad_width

def process_region(wsi, mask, top_left, bottom_right, level, desired_patch_size):
    region_width = int(bottom_right[0] - top_left[0])
    region_height = int(bottom_right[1] - top_left[1])

    downsampled_region = wsi.read_region(top_left, level, (region_width, region_height))
    region_array = np.array(downsampled_region.convert("RGB"))
    mask_region = mask.crop((top_left[0], top_left[1], bottom_right[0], bottom_right[1]))
    mask_region_array = np.array(mask_region.resize((region_width, region_height)))

    image_pad_height, image_pad_width = calculate_padding(region_array, desired_patch_size)
    mask_pad_height, mask_pad_width = calculate_padding(mask_region_array, desired_patch_size)

    region_array_padded = np.pad(region_array, ((0, image_pad_height), (0, image_pad_width), (0, 0)), mode='constant', constant_values=255)
    mask_region_array_padded = np.pad(mask_region_array, ((0, mask_pad_height), (0, mask_pad_width)), mode='constant', constant_values=255)

    return region_array_padded, mask_region_array_padded

def crop_patches(wsi_path, mask_path, roi_coords, desired_patch_size=256, desired_patch_number=100):
    wsi = OpenSlide(wsi_path)
    mask = Image.open(mask_path)

    level = find_appropriate_level(wsi, desired_patch_size, desired_patch_number)
    
    # Divide the ROI into smaller regions if necessary
    # For simplicity, here's an example dividing the ROI into four quadrants
    top_left = (int(roi_coords[0][0][0]), int(roi_coords[0][0][1]))
    bottom_right = (int(roi_coords[0][2][0]), int(roi_coords[0][2][1]))
    mid_x = (top_left[0] + bottom_right[0]) // 2
    mid_y = (top_left[1] + bottom_right[1]) // 2

    subregions = [
        (top_left, (mid_x, mid_y)),
        ((mid_x, top_left[1]), (bottom_right[0], mid_y)),
        ((top_left[0], mid_y), (mid_x, bottom_right[1])),
        ((mid_x, mid_y), bottom_right)
    ]

    wsi_filename_part = os.path.basename(wsi_path).split('.')[0]
    base_save_dir = "C:/Users/beyza/Desktop/AIN3007_project/patch_crop/Breast3__ihc/"
    image_dir = os.path.join(base_save_dir, f'image_{wsi_filename_part}')
    mask_dir = os.path.join(base_save_dir, f'mask_{wsi_filename_part}')

    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)

    for subregion in subregions:
        region_array_padded, mask_region_array_padded = process_region(wsi, mask, subregion[0], subregion[1], level, desired_patch_size)
        image_patches = patchify(region_array_padded, (desired_patch_size, desired_patch_size, 3), step=desired_patch_size)
        mask_patches = patchify(mask_region_array_padded, (desired_patch_size, desired_patch_size), step=desired_patch_size)

        for i in range(image_patches.shape[0]):
            for j in range(image_patches.shape[1]):
                image_patch = image_patches[i, j, 0, :, :, :]
                image_patch_img = Image.fromarray(image_patch)
                image_patch_path = os.path.join(image_dir, f'patch_{i}_{j}.png')
                image_patch_img.save(image_patch_path)

                mask_patch = mask_patches[i, j, :, :]
                mask_patch_img = Image.fromarray(mask_patch)
                mask_patch_path = os.path.join(mask_dir, f'patch_{i}_{j}.png')
                mask_patch_img.save(mask_patch_path)


In [30]:
for index, row in df.iloc[120:121].iterrows():
    wsi_path = row["Slides"]
    mask_path = row["mask_path"]
    roi_coords = row["roi_coords"]
    crop_patches(wsi_path, mask_path,roi_coords) 

MemoryError: Unable to allocate 2.85 GiB for an array with shape (54784, 55808) and data type uint8

In [29]:
def find_appropriate_level(wsi, desired_patch_size, desired_patch_number):
    """
    WSI'daki en uygun seviyeyi bulur.
    desired_patch_size: Oluşturulacak patch'in piksel cinsinden boyutu
    desired_patch_number: Oluşturulmak istenen minimum patch sayısı
    """
    for level in range(wsi.level_count - 1, -1, -1):
        width, height = wsi.level_dimensions[level]
        if (width // desired_patch_size) * (height // desired_patch_size) >= desired_patch_number:
            return level
    return 0  


def crop_patches(wsi_path, mask_path, roi_coords, desired_patch_size=256, desired_patch_number=100):
    wsi = OpenSlide(wsi_path)
    mask = Image.open(mask_path)

    level = find_appropriate_level(wsi, desired_patch_size, desired_patch_number)

    if roi_coords:
        # Extracting top-left and bottom-right coordinates
        top_left = roi_coords[0][0]  
        top_left_int = (int(top_left[0]), int(top_left[1]))
        bottom_right = roi_coords[0][2]  

        # Calculating width and height of the ROI
        roi_width = bottom_right[0] - top_left[0]
        roi_height = bottom_right[1] - top_left[1]

        # Adjust ROI dimensions according to the level downsampling
        level_downsample = wsi.level_downsamples[level]
        x, y = int(top_left[0] / level_downsample), int(top_left[1] / level_downsample)
        width, height = int(roi_width / level_downsample), int(roi_height / level_downsample)

        # Extend the ROI if necessary to make it divisible by the patch size
        patch_size = 256
        width_extended = (width // patch_size + (width % patch_size > 0)) * patch_size
        height_extended = (height // patch_size + (height % patch_size > 0)) * patch_size

        downsampled_image = wsi.read_region(top_left_int, level, (width_extended, height_extended))
    else:
        downsampled_image = wsi.read_region((0, 0), level, wsi.level_dimensions[level])

    # Convert the PIL images to numpy arrays
    image_array = np.array(downsampled_image)
    mask_array = np.array(mask)

    # Ensure the mask is resized to match the WSI's downsampled dimensions
    mask_array_resized = np.array(mask.resize(downsampled_image.size))

    # Create patches for both image and mask
    image_patches = patchify(image_array, (patch_size, patch_size, 3), step=patch_size)
    mask_patches = patchify(mask_array_resized, (patch_size, patch_size), step=patch_size)
    
    wsi_filename_part = os.path.basename(wsi_path).split('.')[0]
    
    base_save_dir = "C:/Users/beyza/Desktop/AIN3007_project/patch_crop/Breast2__he/"
    image_dir = os.path.join(base_save_dir, f'image_{wsi_filename_part}')
    mask_dir = os.path.join(base_save_dir, f'mask_{wsi_filename_part}')
    
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)
    
    for i in range(image_patches.shape[0]):
        for j in range(image_patches.shape[1]):
            # Save image patch
            image_patch = image_patches[i, j, 0, :, :, :]
            image_patch_img = Image.fromarray(image_patch)
            image_patch_path = os.path.join(image_dir, f'patch_{i}_{j}.png')
            image_patch_img.save(image_patch_path)

            # Save mask patch
            mask_patch = mask_patches[i, j, :, :]
            mask_patch_img = Image.fromarray(mask_patch)
            mask_patch_path = os.path.join(mask_dir, f'patch_{i}_{j}.png')
            mask_patch_img.save(mask_patch_path)


In [30]:
for index, row in df.iloc[40:80].iterrows():
    wsi_path = row["Slides"]
    mask_path = row["mask_path"]
    roi_coords = row["roi_coords"]
    crop_patches(wsi_path, mask_path, roi_coords) 

In [43]:
for index, row in df.iloc[120:121].iterrows():
    wsi_path = row["Slides"]
    mask_path = row["mask_path"]
    roi_coords = row["roi_coords"]
    top_left = roi_coords[0][0]
    print(top_left)

(4040.67578, 51040.1133)
